In [14]:
import json
import gzip
import pandas as pd


In [15]:
with gzip.open("dataset0.json.gz", mode="rt") as f:
    data = f.read()

In [16]:
import json
import gzip

# Read the JSONL file and process the first line (or iterate for multiple objects)
try:
    with gzip.open("dataset0.json.gz", mode="rt", encoding="utf-8") as f:
        # Assuming the provided data is the first line
        json_string = f.readline()  # Read the first line
        nested_dict = json.loads(json_string)
        print("Successfully converted JSON string to dictionary.")
except FileNotFoundError:
    print("Error: The file 'dataset0.json.gz' was not found. Please check the file path.")
    raise
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    raise
except Exception as e:
    print(f"Error reading file: {e}")
    raise



Successfully converted JSON string to dictionary.


In [17]:
nested_dict.keys()



dict_keys(['ENST00000000233'])

In [18]:
import json
import gzip

# Initialize an empty dictionary to store all data
nested_dict = {}

# Read the JSONL file and process each line
with gzip.open("dataset0.json.gz", mode="rt", encoding="utf-8") as f:
    for line in f:
        line_dict = json.loads(line.strip())
        # Get the transcript_id (ENST key)
        enst_key = next(iter(line_dict), None)
        if enst_key in nested_dict:
            # Merge nested data (e.g., combine data under "244" or other subkeys)
            nested_dict[enst_key].update(line_dict[enst_key])
        else:
            # Add new ENST key and its data
            nested_dict[enst_key] = line_dict[enst_key]




In [19]:
print("\nTop-level ENST keys:", list(nested_dict.keys()))


Top-level ENST keys: ['ENST00000000233', 'ENST00000000412', 'ENST00000002165', 'ENST00000003100', 'ENST00000007516', 'ENST00000005257', 'ENST00000001008', 'ENST00000007390', 'ENST00000007264', 'ENST00000009589', 'ENST00000009041', 'ENST00000005340', 'ENST00000016171', 'ENST00000014930', 'ENST00000002596', 'ENST00000005260', 'ENST00000022615', 'ENST00000011619', 'ENST00000005386', 'ENST00000005286', 'ENST00000037243', 'ENST00000023939', 'ENST00000012443', 'ENST00000013807', 'ENST00000037869', 'ENST00000014914', 'ENST00000044462', 'ENST00000053867', 'ENST00000053468', 'ENST00000054666', 'ENST00000158771', 'ENST00000055077', 'ENST00000054950', 'ENST00000160827', 'ENST00000066544', 'ENST00000078429', 'ENST00000081029', 'ENST00000084795', 'ENST00000085068', 'ENST00000175091', 'ENST00000157812', 'ENST00000162749', 'ENST00000166345', 'ENST00000168216', 'ENST00000169551', 'ENST00000170447', 'ENST00000173527', 'ENST00000175506', 'ENST00000199389', 'ENST00000177742', 'ENST00000179259', 'ENST000

In [20]:
if nested_dict:
    first_enst_key = list(nested_dict.keys())[0]
    print(f"Keys under '{first_enst_key}':", list(nested_dict[first_enst_key].keys()))
    first_subkey = list(nested_dict[first_enst_key].keys())[0]
    print(f"Keys under '{first_subkey}':", list(nested_dict[first_enst_key][first_subkey].keys()))
    print(f"First data point under 'AAGACCA':", nested_dict[first_enst_key][first_subkey]["AAGACCA"][0])
    print(f"Number of data points under 'AAGACCA':", len(nested_dict[first_enst_key][first_subkey]["AAGACCA"]))

Keys under 'ENST00000000233': ['244', '261', '316', '332', '368', '404', '431', '440', '471', '539', '571', '575', '665', '696', '752', '757', '765', '913']
Keys under '244': ['AAGACCA']
First data point under 'AAGACCA': [0.00299, 2.06, 125.0, 0.0177, 10.4, 122.0, 0.0093, 10.9, 84.1]
Number of data points under 'AAGACCA': 185


In [21]:
data_info = pd.read_csv('data.info.labelled')
print(data_info)

                gene_id    transcript_id  transcript_position  label
0       ENSG00000004059  ENST00000000233                  244      0
1       ENSG00000004059  ENST00000000233                  261      0
2       ENSG00000004059  ENST00000000233                  316      0
3       ENSG00000004059  ENST00000000233                  332      0
4       ENSG00000004059  ENST00000000233                  368      0
...                 ...              ...                  ...    ...
121833  ENSG00000167747  ENST00000641834                 1348      1
121834  ENSG00000167747  ENST00000641834                 1429      0
121835  ENSG00000167747  ENST00000641834                 1531      1
121836  ENSG00000167747  ENST00000641834                 1537      0
121837  ENSG00000167747  ENST00000641834                 1693      0

[121838 rows x 4 columns]


In [22]:
import json
import gzip
import pandas as pd
import ujson as json  # faster JSON parsing

# Read the CSV file
data_info = pd.read_csv('data.info.labelled')
print("DataFrame from data.info.labelled:")
print(data_info.head())  # Print first few rows for verification


# pairs every transcript_id to gene_id for quick look up
gene_id_lookup = dict(zip(data_info['transcript_id'], data_info['gene_id']))
# for pair (transcript_id, gene_id) to label
label_lookup = dict(zip(zip(data_info['transcript_id'], data_info['transcript_position'].astype(str)), data_info['label']))

# Initialize the new nested dictionary
new_dict = {}

# Read the JSONL file and process each line
with gzip.open("dataset0.json.gz", mode="rt", encoding="utf-8") as f:
    for line in f:
        #gets a line in json
        line_dict = json.loads(line.strip())   
        #finds transcript ID and makes it a key
        transcript_id = next(iter(line_dict), None)
        #finds gene_id
        gene_id = gene_id_lookup.get(transcript_id)
        if gene_id not in new_dict:
            new_dict[gene_id] = {}
        # makes transcript_id in new_dict if not present
        if transcript_id not in new_dict[gene_id]:
            new_dict[gene_id][transcript_id] = {}               
        # inputs each transcript_position in the JSON data
        for transcript_position, position_data in line_dict[transcript_id].items():
            # inputs transcript_position in new_dict if not inside
            if transcript_position not in new_dict[gene_id][transcript_id]:
                new_dict[gene_id][transcript_id][transcript_position] = {} 
            # finds label for this (transcript_id, transcript_position) pair
            label = label_lookup.get((transcript_id, transcript_position), None) 
            # Process each sequence (e.g., "AAGACCA")
            for sequence, data_points in position_data.items():
                # Initialize sequence in new_dict if not present
                if sequence not in new_dict[gene_id][transcript_id][transcript_position]:
                    new_dict[gene_id][transcript_id][transcript_position][sequence] = []
                        
                # Prepend the label to each data point
                modified_data_points = [[label] + data_point for data_point in data_points]
                new_dict[gene_id][transcript_id][transcript_position][sequence] = modified_data_points
                



DataFrame from data.info.labelled:
           gene_id    transcript_id  transcript_position  label
0  ENSG00000004059  ENST00000000233                  244      0
1  ENSG00000004059  ENST00000000233                  261      0
2  ENSG00000004059  ENST00000000233                  316      0
3  ENSG00000004059  ENST00000000233                  332      0
4  ENSG00000004059  ENST00000000233                  368      0


In [23]:
# Verify the structure
print("\nTop-level gene_id keys:", list(new_dict.keys()))



Top-level gene_id keys: ['ENSG00000004059', 'ENSG00000003056', 'ENSG00000001036', 'ENSG00000001630', 'ENSG00000004779', 'ENSG00000006451', 'ENSG00000004478', 'ENSG00000007520', 'ENSG00000007376', 'ENSG00000008988', 'ENSG00000010270', 'ENSG00000004975', 'ENSG00000014919', 'ENSG00000013583', 'ENSG00000002587', 'ENSG00000006453', 'ENSG00000078668', 'ENSG00000010017', 'ENSG00000005175', 'ENSG00000006118', 'ENSG00000034713', 'ENSG00000022277', 'ENSG00000011485', 'ENSG00000012061', 'ENSG00000035141', 'ENSG00000013588', 'ENSG00000041357', 'ENSG00000030582', 'ENSG00000048544', 'ENSG00000049245', 'ENSG00000072849', 'ENSG00000049541', 'ENSG00000049449', 'ENSG00000079616', 'ENSG00000004897', 'ENSG00000088256', 'ENSG00000061794', 'ENSG00000063177', 'ENSG00000063241', 'ENSG00000068697', 'ENSG00000013275', 'ENSG00000067182', 'ENSG00000071539', 'ENSG00000072506', 'ENSG00000075336', 'ENSG00000075975', 'ENSG00000066583', 'ENSG00000070669', 'ENSG00000086232', 'ENSG00000074071', 'ENSG00000078237', 'ENSG

In [24]:
if new_dict:
    first_gene_id = list(new_dict.keys())[0]
    print(f"Transcript_ids under '{first_gene_id}':", list(new_dict[first_gene_id].keys()))
    first_transcript_id = list(new_dict[first_gene_id].keys())[0]
    print(f"Transcript_positions under '{first_transcript_id}':", list(new_dict[first_gene_id][first_transcript_id].keys()))
    first_position = list(new_dict[first_gene_id][first_transcript_id].keys())[0]
    print(f"Sequences under '{first_position}':", list(new_dict[first_gene_id][first_transcript_id][first_position].keys()))
    first_sequence = list(new_dict[first_gene_id][first_transcript_id][first_position].keys())[0]
    print(f"First data point under '{first_sequence}':", new_dict[first_gene_id][first_transcript_id][first_position][first_sequence][0])
    print(f"Number of data points under '{first_sequence}':", len(new_dict[first_gene_id][first_transcript_id][first_position][first_sequence]))

Transcript_ids under 'ENSG00000004059': ['ENST00000000233']
Transcript_positions under 'ENST00000000233': ['244', '261', '316', '332', '368', '404', '431', '440', '471', '539', '571', '575', '665', '696', '752', '757', '765', '913']
Sequences under '244': ['AAGACCA']
First data point under 'AAGACCA': [0, 0.00299, 2.06, 125.0, 0.0177, 10.4, 122.0, 0.0093, 10.9, 84.1]
Number of data points under 'AAGACCA': 185


Making them into a normal data frame for the first 100 genes.

In [29]:
rows = []
gene_ids = list(new_dict.keys())[:] 

for gene_id in gene_ids:
    for transcript_id in new_dict[gene_id]:
        for transcript_position in new_dict[gene_id][transcript_id]:
            for sequence, data_points in new_dict[gene_id][transcript_id][transcript_position].items():
                for data_point in data_points:
                    # Each data_point is [label, feature_1, ..., feature_9]
                    row = [gene_id, transcript_id, transcript_position, sequence] + data_point
                    rows.append(row)

# Create DataFrame
columns = ['gene_id', 'transcript_id', 'transcript_position', 'sequence', 'label'] + [f'feature_{i+1}' for i in range(9)]
df = pd.DataFrame(rows, columns=columns)

# Save DataFrame to CSV
df.to_csv('Full_genes.csv', index=False)
print("\nDataFrame saved to 'Full_genes.csv'")



DataFrame saved to 'Full_genes.csv'


In [28]:
num_genes = 50


# Convert nested dictionary to DataFrame for first 50 genes
rows = []
gene_ids = list(new_dict.keys())[:num_genes]  # Take first 50 gene_ids (or all if fewer)

for gene_id in gene_ids:
    for transcript_id in new_dict[gene_id]:
        for transcript_position in new_dict[gene_id][transcript_id]:
            for sequence, data_points in new_dict[gene_id][transcript_id][transcript_position].items():
                for data_point in data_points:
                    # Each data_point is [label, feature_1, ..., feature_9]
                    row = [gene_id, transcript_id, transcript_position, sequence] + data_point
                    rows.append(row)

# Create DataFrame
columns = ['gene_id', 'transcript_id', 'transcript_position', 'sequence', 'label'] + [f'feature_{i+1}' for i in range(9)]
df = pd.DataFrame(rows, columns=columns)

# Save DataFrame to CSV
df.to_csv('50_genes.csv', index=False)
print("\nDataFrame saved to '50_genes.csv'")


DataFrame saved to '50_genes.csv'
